## 1. Importación de Librerías
Importamos las librerías necesarias para el análisis y procesamiento.

In [ ]:
# Librerías básicas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Librerías para procesamiento de señales
from scipy import signal
from scipy.fft import fft, fftfreq

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

print("Librerías importadas exitosamente")

## 2. Configuración y Parámetros
Definimos los parámetros principales del sistema.

In [ ]:
# Parámetros del sistema
fs = 1000  # Frecuencia de muestreo (Hz)
T = 1.0    # Duración de la señal (s)
N = int(fs * T)  # Número de muestras

# Vector de tiempo
t = np.linspace(0, T, N, endpoint=False)

print(f"Frecuencia de muestreo: {fs} Hz")
print(f"Duración: {T} s")
print(f"Número de muestras: {N}")

## 3. Generación de Señales
Creamos las señales de prueba para el análisis.

In [ ]:
# Generar señales de ejemplo
f1 = 50   # Frecuencia 1 (Hz)
f2 = 120  # Frecuencia 2 (Hz)

# Señal compuesta
signal1 = np.sin(2 * np.pi * f1 * t)
signal2 = 0.5 * np.sin(2 * np.pi * f2 * t)
noise = 0.1 * np.random.randn(len(t))

# Señal total
signal_total = signal1 + signal2 + noise

print("Señales generadas")
print(f"Componente 1: {f1} Hz")
print(f"Componente 2: {f2} Hz")
print(f"Ruido agregado: σ = 0.1")

## 4. Visualización en el Dominio del Tiempo
Graficamos las señales en el dominio temporal.

In [ ]:
# Crear subplots
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Componente 1
axes[0].plot(t[:200], signal1[:200], 'b-', linewidth=2)
axes[0].set_title(f'Señal 1: {f1} Hz')
axes[0].set_ylabel('Amplitud')
axes[0].grid(True)

# Componente 2
axes[1].plot(t[:200], signal2[:200], 'r-', linewidth=2)
axes[1].set_title(f'Señal 2: {f2} Hz')
axes[1].set_ylabel('Amplitud')
axes[1].grid(True)

# Señal total
axes[2].plot(t[:200], signal_total[:200], 'g-', linewidth=1)
axes[2].set_title('Señal Total (con ruido)')
axes[2].set_xlabel('Tiempo (s)')
axes[2].set_ylabel('Amplitud')
axes[2].grid(True)

plt.tight_layout()
plt.show()

## 5. Análisis en el Dominio de la Frecuencia
Aplicamos la Transformada de Fourier para analizar el espectro de frecuencias.

In [ ]:
# Calcular FFT
fft_signal = fft(signal_total)
freqs = fftfreq(N, 1/fs)

# Solo frecuencias positivas
positive_freqs = freqs[:N//2]
fft_magnitude = 2.0/N * np.abs(fft_signal[:N//2])

# Graficar espectro
plt.figure(figsize=(12, 6))
plt.plot(positive_freqs, fft_magnitude, 'b-', linewidth=2)
plt.title('Espectro de Frecuencias - FFT')
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Magnitud')
plt.xlim(0, 200)
plt.grid(True)
plt.show()

# Encontrar picos principales
peaks, _ = signal.find_peaks(fft_magnitude, height=0.1)
peak_freqs = positive_freqs[peaks]
peak_mags = fft_magnitude[peaks]

print("Frecuencias detectadas:")
for freq, mag in zip(peak_freqs, peak_mags):
    print(f"  {freq:.1f} Hz: {mag:.3f}")

## 6. Diseño de Filtros
Implementamos filtros digitales para procesamiento de señales.

In [ ]:
# Diseño de filtro pasa-bajos
cutoff_freq = 80  # Frecuencia de corte (Hz)
nyquist = fs / 2
normalized_cutoff = cutoff_freq / nyquist

# Filtro Butterworth de orden 4
b, a = signal.butter(4, normalized_cutoff, btype='low')

# Aplicar filtro
filtered_signal = signal.filtfilt(b, a, signal_total)

# Respuesta en frecuencia del filtro
w, h = signal.freqz(b, a, worN=8000)
freq_response = w * fs / (2 * np.pi)

# Graficar respuesta del filtro
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Magnitud
ax1.plot(freq_response, 20 * np.log10(abs(h)), 'b-', linewidth=2)
ax1.set_title('Respuesta en Frecuencia del Filtro')
ax1.set_ylabel('Magnitud (dB)')
ax1.set_xlim(0, 200)
ax1.grid(True)
ax1.axvline(cutoff_freq, color='r', linestyle='--', label=f'Fc = {cutoff_freq} Hz')
ax1.legend()

# Comparación señal original vs filtrada
ax2.plot(t[:500], signal_total[:500], 'b-', alpha=0.7, label='Original')
ax2.plot(t[:500], filtered_signal[:500], 'r-', linewidth=2, label='Filtrada')
ax2.set_title('Señal Original vs Filtrada')
ax2.set_xlabel('Tiempo (s)')
ax2.set_ylabel('Amplitud')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

## 7. Análisis Estadístico
Realizamos análisis estadístico de las señales.

In [ ]:
# Estadísticas descriptivas
stats_original = {
    'Media': np.mean(signal_total),
    'Desviación Estándar': np.std(signal_total),
    'RMS': np.sqrt(np.mean(signal_total**2)),
    'Valor Máximo': np.max(signal_total),
    'Valor Mínimo': np.min(signal_total)
}

stats_filtered = {
    'Media': np.mean(filtered_signal),
    'Desviación Estándar': np.std(filtered_signal),
    'RMS': np.sqrt(np.mean(filtered_signal**2)),
    'Valor Máximo': np.max(filtered_signal),
    'Valor Mínimo': np.min(filtered_signal)
}

# Crear DataFrame para comparación
df_stats = pd.DataFrame({
    'Señal Original': stats_original,
    'Señal Filtrada': stats_filtered
})

print("Estadísticas Comparativas:")
print(df_stats.round(4))

# Histograma de amplitudes
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(signal_total, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.title('Distribución de Amplitudes - Señal Original')
plt.xlabel('Amplitud')
plt.ylabel('Frecuencia')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(filtered_signal, bins=50, alpha=0.7, color='red', edgecolor='black')
plt.title('Distribución de Amplitudes - Señal Filtrada')
plt.xlabel('Amplitud')
plt.ylabel('Frecuencia')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Resultados y Conclusiones
Resumen de los resultados obtenidos.

### Resultados Principales:

1. **Señales Generadas:**
   - Se generaron exitosamente señales sinusoidales de 50 Hz y 120 Hz
   - Se agregó ruido gaussiano para simular condiciones reales

2. **Análisis Frecuencial:**
   - La FFT identificó correctamente los componentes frecuenciales
   - Los picos en 50 Hz y 120 Hz son claramente visibles

3. **Filtrado:**
   - El filtro pasa-bajos eliminó efectivamente las frecuencias altas
   - Se preservó la componente de 50 Hz
   - Se atenuó significativamente la componente de 120 Hz

### Conclusiones:

- Las técnicas de procesamiento digital implementadas son efectivas
- El filtro Butterworth mostró buen desempeño
- Los métodos estadísticos confirman la mejora en la relación señal/ruido

### Trabajo Futuro:

- [ ] Implementar filtros adaptativos
- [ ] Análizar diferentes tipos de ventanas para FFT
- [ ] Estudiar filtros FIR vs IIR
- [ ] Optimizar para implementación en tiempo real

---
## Referencias

1. Oppenheim, A. V., & Schafer, R. W. (2010). *Discrete-Time Signal Processing*.
2. Proakis, J. G., & Manolakis, D. G. (2007). *Digital Signal Processing*.
3. Documentación de SciPy: https://docs.scipy.org/

---
*Notebook creado el 27 de enero de 2026 para TAREA-2-TDSE*